In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_hub as hub
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt') 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
df = pd.read_csv("/Users/nikhi/Desktop/sentiment_analysis/IMDB_Dataset/IMDB_Dataset.csv", encoding='ISO-8859-1')

In [5]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
df = df.drop_duplicates()

In [7]:
df = df.dropna()

In [8]:
def remove_stopwords(text):
    text = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word.isalpha() and word not in stop_words]
    return ' '.join(text)


In [9]:
# Convert sentiment labels to numeric
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})


In [10]:
# Redefine labels after conversion
x = df['review'].apply(remove_stopwords)
y = df['sentiment']


In [11]:
max_word = 20000
max_length = 200

In [12]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=max_word)
tokenizer.fit_on_texts(x)

In [13]:
sequences = tokenizer.texts_to_sequences(x)
x = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max_length)


In [14]:
# model parameter
vocab_size = 1000
embedding_dim = 128
max_length = 200
oov_tok = "<OOV>"
training_portion = 0.7
BATCH_SIZE = 32


In [15]:
train_size = int(df.shape[0] * training_portion)
train_sentences, test_sentences, train_labels, test_labels = train_test_split(x, y, test_size=1-training_portion ,random_state=42)


In [16]:
#load embedding from tensorflow hub
embed = hub.load("https://tfhub.dev/google/nnlm-en-dim128/1")


In [ ]:
# # define the CNN model

# model = keras.Sequential([
#     keras.layers.Embedding(input_dim=max_word, output_dim=embedding_dim,input_shape=(max_length,)),
#     keras.layers.Conv1D(filters=32, kernel_size=6, padding='same', activation='relu'),
#     keras.layers.MaxPooling1D(pool_size=2),
#     # keras.layers.Flatten(),
#     keras.layers.Dropout(0.10),

#     keras.layers.Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'),
#     keras.layers.MaxPooling1D(pool_size=2),
#     # keras.layers.Flatten(),
#     keras.layers.Dropout(0.15),
    

#     keras.layers.Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'),
#     keras.layers.MaxPooling1D(pool_size=2),
#     keras.layers.Dropout(0.20),

#     keras.layers.Flatten(),

#     # keras.layers.Conv1D(128, 5, activation='relu'),
#     # keras.layers.GlobalMaxPooling1D(),
#     keras.layers.Dense(256, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

C:\Users\nikhi\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 200, 32)        │        24,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 100, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 100, 64)        │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 50, 64)         │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       409,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,011,233 (11.49 MB)

 Trainable params: 3,011,233 (11.49 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# model = keras.Sequential([
#     # Embedding layer remains the same
#     keras.layers.Embedding(input_dim=max_word, output_dim=embedding_dim, input_shape=(max_length,)),
    
#     # Increase complexity and depth of convolutional layers
#     keras.layers.Conv1D(filters=64, kernel_size=6, padding='same', activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPooling1D(pool_size=2),
#     keras.layers.Dropout(0.15),

#     keras.layers.Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPooling1D(pool_size=2),
#     keras.layers.Dropout(0.20),

#     keras.layers.Conv1D(filters=256, kernel_size=3, padding='same', activation='relu'),
#     keras.layers.BatchNormalization(),
#     keras.layers.MaxPooling1D(pool_size=2),
#     keras.layers.Dropout(0.25),

#     # Flatten and add more dense layers
#     keras.layers.Flatten(),
    
#     # Add more dense layers with regularization
#     keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dropout(0.30),
    
#     keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
#     keras.layers.BatchNormalization(),
#     keras.layers.Dropout(0.25),
    
#     # Output layer
#     keras.layers.Dense(1, activation='sigmoid')
# ])

# # More sophisticated compilation
# model.compile(
#     loss='binary_crossentropy', 
#     optimizer=keras.optimizers.Adam(learning_rate=0.0005),
#     metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall(), keras.metrics.AUC()]
# )
# model.summary()

C:\Users\nikhi\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 200, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 200, 64)        │        49,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 200, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 100, 128)       │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 100, 128)       │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 50, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 50, 256)        │        98,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 50, 256)        │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 25, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 25, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │     3,277,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,154,433 (23.48 MB)

 Trainable params: 6,152,001 (23.47 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [38]:
model = keras.Sequential([
    # Keep the original embedding layer
    keras.layers.Embedding(input_dim=max_word, output_dim=embedding_dim, input_shape=(max_length,)),
    
    # Slightly modify convolutional layers
    keras.layers.Conv1D(filters=48, kernel_size=6, padding='same', activation='relu'),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Dropout(0.12),  # Slightly adjusted dropout

    keras.layers.Conv1D(filters=96, kernel_size=4, padding='same', activation='relu'),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Dropout(0.16),  # Slightly adjusted dropout

    keras.layers.Conv1D(filters=128, kernel_size=2, padding='same', activation='relu'),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.Dropout(0.18),

    keras.layers.Flatten(),

    # Modest dense layers with careful regularization
    keras.layers.Dense(
        64, 
        activation='relu', 
        kernel_regularizer=keras.regularizers.l1_l2(l1=1e-5, l2=1e-4)
    ),
    keras.layers.Dropout(0.22),
    
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile with adaptive learning rate
model.compile(
    loss='binary_crossentropy', 
    optimizer=keras.optimizers.Adam(learning_rate=0.001),  # Restored to a standard learning rate
    metrics=['accuracy']
)
model.summary()

C:\Users\nikhi\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 200, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 200, 48)        │        36,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_9 (MaxPooling1D)  │ (None, 100, 48)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 100, 48)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 100, 96)        │        18,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_10 (MaxPooling1D) │ (None, 50, 96)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 50, 96)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_11 (Conv1D)              │ (None, 50, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_11 (MaxPooling1D) │ (None, 25, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 25, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 3200)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,845,073 (10.85 MB)

 Trainable params: 2,845,073 (10.85 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# import tensorflow as tf
# import keras
# from keras import layers, regularizers

# def create_enhanced_model(max_word, embedding_dim, max_length):
#     """
#     Create an enhanced Convolutional Neural Network model 
#     with advanced techniques to improve accuracy.
    
#     Key Enhancements:
#     1. More sophisticated convolutional architecture
#     2. Advanced regularization techniques
#     3. Batch normalization
#     4. Adjusted learning rates and optimizers
#     5. Deeper network with carefully tuned parameters
#     """
#     model = keras.Sequential([
#         # Enhanced Embedding Layer
#         layers.Embedding(
#             input_dim=max_word, 
#             output_dim=embedding_dim, 
#             input_shape=(max_length,),
#             embeddings_regularizer=regularizers.l2(1e-5)  # Added embedding regularization
#         ),
        
#         # First Convolutional Block with Enhanced Features
#         layers.Conv1D(
#             filters=12,  # Increased filters 
#             kernel_size=7, 
#             padding='same', 
#             activation='relu',
#             kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
#         ),
#         layers.BatchNormalization(),  # Added batch normalization
#         layers.MaxPooling1D(pool_size=2),
#         layers.SpatialDropout1D(0.15),  # Switched to Spatial Dropout
        
#         # Second Convolutional Block with Increased Complexity
#         layers.Conv1D(
#             filters=24, 
#             kernel_size=5, 
#             padding='same', 
#             activation='relu',
#             kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
#         ),
#         layers.BatchNormalization(),
#         layers.MaxPooling1D(pool_size=2),
#         layers.SpatialDropout1D(0.18),
        
#         # Third Convolutional Block with Deeper Features
#         layers.Conv1D(
#             filters=36, 
#             kernel_size=3, 
#             padding='same', 
#             activation='relu',
#             kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4)
#         ),
#         layers.BatchNormalization(),
#         layers.MaxPooling1D(pool_size=2),
#         layers.SpatialDropout1D(0.2),
        
#         # Global Average Pooling for More Robust Feature Representation
#         layers.GlobalAveragePooling1D(),
        
#         # Dense Layers with Advanced Regularization
#         layers.Dense(
#             128, 
#             activation='relu',
#             kernel_regularizer=regularizers.l1_l2(l1=1e-6, l2=1e-5)
#         ),
#         layers.BatchNormalization(),
#         layers.Dropout(0.25),
        
#         layers.Dense(
#             64, 
#             activation='relu',
#             kernel_regularizer=regularizers.l1_l2(l1=1e-6, l2=1e-5)
#         ),
#         layers.BatchNormalization(),
#         layers.Dropout(0.22),
        
#         # Output Layer
#         layers.Dense(1, activation='sigmoid')
#     ])
    
#     # Advanced Optimizer with Learning Rate Scheduling
#     initial_learning_rate = 0.001
#     lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#         initial_learning_rate,
#         decay_steps=100000,
#         decay_rate=0.96,
#         staircase=True
#     )
    
#     optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
#     # Compile with Advanced Configurations
#     model.compile(
#         loss='binary_crossentropy',
#         optimizer=optimizer,
#         metrics=['accuracy', 
#                  tf.keras.metrics.Precision(), 
#                  tf.keras.metrics.Recall()]
#     )
    
#     return model

# # Example Usage
# model = create_enhanced_model(max_word=20000, embedding_dim=200, max_length=250)
# model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_8 (Embedding)         │ (None, 250, 200)       │     4,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_24 (Conv1D)              │ (None, 250, 12)        │        16,812 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 250, 12)        │            48 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_24 (MaxPooling1D) │ (None, 125, 12)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_12            │ (None, 125, 12)        │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_25 (Conv1D)              │ (None, 125, 24)        │         1,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 125, 24)        │            96 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_25 (MaxPooling1D) │ (None, 62, 24)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_13            │ (None, 62, 24)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_26 (Conv1D)              │ (None, 62, 36)         │         2,628 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 62, 36)         │           144 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_26 (MaxPooling1D) │ (None, 31, 36)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_14            │ (None, 31, 36)         │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_4      │ (None, 36)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 128)            │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 4,035,017 (15.39 MB)

 Trainable params: 4,034,489 (15.39 MB)

 Non-trainable params: 528 (2.06 KB)

In [109]:
model = keras.Sequential([
    keras.layers.Embedding(input_dim=max_word, output_dim=embedding_dim,input_shape=(max_length,)),
    keras.layers.Conv1D(128, 5,padding='same', activation='relu'),
    keras.layers.GlobalMaxPooling1D(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_19 (Embedding)        │ (None, 200, 128)       │     2,560,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_42 (Conv1D)              │ (None, 200, 128)       │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_9          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_38 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_39 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,650,369 (10.11 MB)

 Trainable params: 2,650,369 (10.11 MB)

 Non-trainable params: 0 (0.00 B)

In [106]:
#train model
num_epochs = 5

history = model.fit(train_sentences, train_labels, epochs=num_epochs,batch_size=BATCH_SIZE, validation_split=0.1)

Epoch 1/5
977/977 ━━━━━━━━━━━━━━━━━━━━ 46s 47ms/step - accuracy: 0.9991 - loss: 0.0029 - val_accuracy: 0.8539 - val_loss: 0.7166
Epoch 2/5
977/977 ━━━━━━━━━━━━━━━━━━━━ 46s 47ms/step - accuracy: 0.9910 - loss: 0.0265 - val_accuracy: 0.8516 - val_loss: 0.8341
Epoch 3/5
977/977 ━━━━━━━━━━━━━━━━━━━━ 46s 47ms/step - accuracy: 0.9963 - loss: 0.0099 - val_accuracy: 0.8778 - val_loss: 0.7186
Epoch 4/5
977/977 ━━━━━━━━━━━━━━━━━━━━ 45s 46ms/step - accuracy: 0.9998 - loss: 8.1412e-04 - val_accuracy: 0.8732 - val_loss: 0.7792
Epoch 5/5
977/977 ━━━━━━━━━━━━━━━━━━━━ 45s 46ms/step - accuracy: 0.9998 - loss: 7.7254e-04 - val_accuracy: 0.8701 - val_loss: 0.8626


In [80]:
# model.save_weights('CNN_model-4_IMDB_Dataset.weights.h5')

In [101]:
y_pred = model.predict(test_sentences)

465/465 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step


In [102]:
y_pred_binary = (y_pred > 0.5).astype(int)

In [103]:
cm = confusion_matrix(test_labels, y_pred_binary)

In [104]:
class_names = ['Negative', 'Positive']
report = classification_report(test_labels, y_pred_binary, target_names=class_names)
print("Classification Report:\n", report)

Classification Report:
               precision    recall  f1-score   support

    Negative       0.86      0.90      0.88      7404
    Positive       0.90      0.86      0.88      7471

    accuracy                           0.88     14875
   macro avg       0.88      0.88      0.88     14875
weighted avg       0.88      0.88      0.88     14875

